In [4]:
import numpy as np

from nems import Model

x = np.random.rand(10000, 18)  # fake spectrogram
# TODO: something still isn't matching up right when multiple outputs are used,
#       more testing needed.
m = Model.from_keywords('wc.18x4.g-fir.4x15x3')
# TODO: currently all the sample/mean methods still return the samples even if
#       inplace=True (the default for model and layer methods). Did that so that
#       other code wouldn't need to check if something got returned or not, but
#       maybe it would be more intuitive if nothing got returned. Look at pandas
#       code again, see what their default is when inplace=True.
_ = m.sample_from_priors()
m

Model()
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
.layers:

GaussianWeightChannels(shape=(18, 4))
.parameters:

Parameter(name=mean, shape=(4,))
----------------
.prior:     Normal(μ=array(4), σ=array(4))
.bounds:    (0, 1)
.is_frozen: False
.values:
[0.31576161 0.81496273 0.47758842 0.83612172]
----------------
Index: 0
----------------

Parameter(name=sd, shape=(4,))
----------------
.prior:     HalfNormal(σ=array(4))
.bounds:    (1.1920929e-07, inf)
.is_frozen: False
.values:
[0.1604208  0.24377576 0.52829346 0.19462368]
----------------
Index: 0
----------------

FIR(shape=(4, 15, 3))
.parameters:

Parameter(name=coefficients, shape=(4, 15, 3))
----------------
.prior:     Normal(μ=array(4, 15, 3), σ=array(4, 15, 3))
.bounds:    (-inf, inf)
.is_frozen: False
.values:
[[[-1.20977863e+00 -8.48839360e-01  4.97012781e-03]
  [ 1.00221047e+00 -2.46081265e-01 -7.69008580e-01]
  [ 7.08640844e-01 -7.36812596e-01  6.82351514e-01]
  [-3.91270197e-01 -5.62635642e-01 -2.5

In [21]:
wc = m.layers[0]
fir = m.layers[1]
# TODO: have wc, fir not return list if only one?
#       wastes operations in Model.evaluate but would
#       make testing of individual layers more intuitive
y = wc.evaluate(x)[0]
z1 = fir.evaluate(y)[0]
z2 = fir.old_evaluate(y)[0]
np.sum(np.isnan(z1)) + np.sum(np.isnan(z2))  # nans always !=

0

In [25]:
np.sum(np.round(z1[:,0][:,np.newaxis], 10) != np.round(z2, 10))

10000

In [ ]:
import numpy as np

from nems.layers.filter import get_zi

# TODO: looks like this just always returns zeros with 1 less entry than filter?
x = np.random.rand(1230492,)
coefficients = np.random.rand(17,)
zi = get_zi(coefficients, x)
zi.shape